In [1]:
import torch
import torch.nn.functional as F
from tqdm import tqdm


import torch

from models.resnet_simclr import ResNetSimCLR
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, TensorDataset



In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
# load model
classification_out_dim = 100
model = ResNetSimCLR(base_model='resnet18', out_dim=128, classification_out_dim=classification_out_dim)
model.load_state_dict(torch.load('/home/zixiwang/mtlsimclr/SimCLR/runs/Jan08_00-22-39_deeplearning-1-vm/checkpoint_0100.pth.tar')['state_dict'])
model = model.to(device)

In [4]:
# FGSM
# load adversarial images
path = '/home/zixiwang/mtlsimclr/SimCLR/adversarial_data/cifar100_fgsm.pt'
adv_images, adv_labels = torch.load(path)
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_fgsm = DataLoader(adv_data, batch_size=128, shuffle=False)

# evluate  robustness result
model.eval()

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in adv_loader_fgsm:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Robust accuracy under FGSM attack: %.2f %%' % (100 * float(correct) / total))

Robust accuracy under FGSM attack: 33.27 %


In [5]:
# PGD-7
# load adversarial images
path = '/home/zixiwang/mtlsimclr/SimCLR/adversarial_data/cifar100_pgd.pt'
adv_images, adv_labels = torch.load(path)
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_7 = DataLoader(adv_data, batch_size=128, shuffle=False)

# evluate  robustness result
model.eval()

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in adv_loader_7:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Robust accuracy under PGD-7 attack: %.2f %%' % (100 * float(correct) / total))

Robust accuracy under PGD-7 attack: 42.71 %


In [6]:
# PGD-20
# load adversarial images
path = '/home/zixiwang/mtlsimclr/SimCLR/adversarial_data/cifar100_pgd_20.pt'
adv_images, adv_labels = torch.load(path)
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_20 = DataLoader(adv_data, batch_size=128, shuffle=False)

# evluate  robustness result
model.eval()

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in adv_loader_20:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Robust accuracy under PGD-20 attack: %.2f %%' % (100 * float(correct) / total))

Robust accuracy under PGD-20 attack: 33.99 %


In [7]:
# PGD-50
# load adversarial images
path = '/home/zixiwang/mtlsimclr/SimCLR/adversarial_data/cifar100_pgd_50.pt'
adv_images, adv_labels = torch.load(path)
adv_data = TensorDataset(adv_images, adv_labels)
adv_loader_50 = DataLoader(adv_data, batch_size=128, shuffle=False)

# evluate  robustness result
model.eval()

correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in adv_loader_50:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
print('Robust accuracy under PGD-50 attack: %.2f %%' % (100 * float(correct) / total))

Robust accuracy under PGD-50 attack: 34.01 %


In [8]:
batch_size = 24

cifar100_test  = datasets.CIFAR100(root='./data', train=False,
                              download=True, transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(cifar100_test,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [9]:
# Evaluate natural accuracy
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        prediction, features = model(images)
        _, predicted = torch.max(prediction.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'mtl-simclr-cifar100.pth')

Accuracy of the model on the test images: 44.56 %
